# Section 7, Lecture 6: Project: Build a Multi-Agent System for Business Analysis

Welcome to the final lecture of Section 7! You’ve learned about multi-agent systems, explored AutoGen, and got hands-on with CrewAI to automate customer support. Now, it’s time to put it all together in a capstone project. In this notebook, you’ll build a multi-agent system using CrewAI to analyze business data and generate a sales report—a real-world task that can save your team hours and help you make smarter decisions.

## Step 1: Installing CrewAI
In this cell, we’re checking if CrewAI is installed. If you see a version number, you’re good to go! If not, follow the instructions to install it using `uv pip install crewai crewai-tools` in your terminal. We’re using `uv` as our package manager, which we set up in Section 2, for faster and more reliable installations.

In [1]:
# Step 1: Install CrewAI
# If you haven’t installed CrewAI yet, run this in your terminal (not here):
# uv pip install crewai crewai-tools
# Mac/Linux: source .venv/bin/activate
# Windows: .venv\Scripts\Activate
try:
    import crewai
    print("CrewAI is installed! Version:", crewai.__version__)
except ImportError:
    print("CrewAI not found. Please install it using the command above.")

CrewAI is installed! Version: 0.102.0


## Step 2: Loading the OpenAI API Key
We’re loading your OpenAI API key from the `.env` file to use GPT-4 as the brain for our agents. If you see “API key loaded successfully!” you’re ready to proceed. If not, ensure your `.env` file has `OPENAI_API_KEY=your-key-here`. Never share your API key publicly!

In [2]:
# Step 2: Load your OpenAI API key
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Make sure your .env file has OPENAI_API_KEY set.")

API key loaded successfully!


## Step 3: Setting Up CrewAI Agents
We’re creating three agents:
- **Data Collector Agent**: Gathers sales data.
- **Analyst Agent**: Analyzes trends in the data.
- **Report Writer Agent**: Writes the sales report.

Each agent has a role, goal, and backstory, powered by GPT-4 using your API key.

In [3]:
# Step 3: Set up CrewAI agents
from crewai import Agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    api_key=openai_api_key
)

data_collector = Agent(
    role="Data Collector",
    goal="Gather sales data for analysis",
    backstory="You’re an expert at collecting and organizing data for business insights.",
    llm=llm,
    verbose=True
)

analyst = Agent(
    role="Data Analyst",
    goal="Analyze sales data and identify key trends or issues",
    backstory="You’re a skilled analyst with a keen eye for trends and insights.",
    llm=llm,
    verbose=True
)

report_writer = Agent(
    role="Report Writer",
    goal="Write a clear and concise sales report with recommendations",
    backstory="You’re a talented writer who creates actionable business reports.",
    llm=llm,
    verbose=True
)
print("Agents created successfully!")

Agents created successfully!


## Step 4: Defining Tasks and Creating a Crew
Assigning structured tasks to each agent and creating a `Crew` object to manage the workflow.

In [4]:
# Step 4: Define tasks for the agents
from crewai import Task, Crew

sales_data = """
Q1 Sales: $50,000 (Electronics: $30,000, Clothing: $20,000)
Q2 Sales: $45,000 (Electronics: $25,000, Clothing: $20,000)
Overall trend: Electronics sales dropped by 16% from Q1 to Q2.
"""

collect_task = Task(
    description=f"Gather the following sales data: {sales_data}. Organize it for analysis.",
    expected_output="A structured summary of the sales data, ready for analysis.",
    agent=data_collector
)

analyze_task = Task(
    description="Analyze the sales data provided by the data collector. Identify key trends and issues.",
    expected_output="A summary of sales trends and issues, e.g., 'Electronics sales dropped 16% from Q1 to Q2.'",
    agent=analyst
)

report_task = Task(
    description="Using the analyst’s summary, write a concise sales report with recommendations for the business.",
    expected_output="A polished sales report with actionable recommendations.",
    agent=report_writer
)

crew = Crew(
    agents=[data_collector, analyst, report_writer],
    tasks=[collect_task, analyze_task, report_task],
    verbose=True
)
print("Tasks and crew set up successfully!")

Tasks and crew set up successfully!


## Step 5: Running the Crew
Execute the multi-agent system and print the final sales report.

In [5]:
# Step 5: Run the crew to complete the tasks
result = crew.kickoff()

print("Final Output:\n", result)

# Agent: Data Collector
## Task: Gather the following sales data: 
Q1 Sales: $50,000 (Electronics: $30,000, Clothing: $20,000)
Q2 Sales: $45,000 (Electronics: $25,000, Clothing: $20,000)
Overall trend: Electronics sales dropped by 16% from Q1 to Q2.
. Organize it for analysis.


# Agent: Data Collector
## Final Answer: 
Sales Data Summary:

- Q1 Total Sales: $50,000 
    - Electronics: $30,000 
    - Clothing: $20,000

- Q2 Total Sales: $45,000 
    - Electronics: $25,000 
    - Clothing: $20,000

- Overall Trend: Electronics sales dropped by 16% from Q1 to Q2. 

This data is now structured for further analysis.


# Agent: Data Analyst
## Task: Analyze the sales data provided by the data collector. Identify key trends and issues.


# Agent: Data Analyst
## Final Answer: 
The overall sales decreased by 10% from Q1 to Q2, reducing from $50,000 to $45,000. This decrease in sales was mainly driven by a reduction in Electronics sales which dropped by 16% from $30,000 in Q1 to $25,000 in Q2.